![ibm cloud logo](./images/ibm-cloud.png)

# Case Study - Data ingestion

The goal of this case study is to put into practice the important concepts from module 1.  We will go through the basic process that begins with refining the business opportunity and ensuring that it is articulated using a scientific thought process.

The business opportunity and case study was first mentioned in Unit 2 of module 1 and like the AAVIAL company itself these data were created for learning purposes.  We will be using the AAVAIL example as a basis for this case study.  [Watch the video again](https://vimeo.com/348708995) if you need a refresher.  You will be gathering data from several provided sources, staging it for quality assurance and saving it in a target destination that is most appropriate.

Watch the video to review the important concepts from the units you just covered and to see an overview of the objectives for this case study.

In [2]:
from IPython.display import IFrame
IFrame('https://player.vimeo.com/video/354996550', width=600,height=400)

## Case study overall objectives

1. Gather all relevant data from the sources of provided data
2. Implement several checks for quality assurance 
3. Take the initial steps towards automation of the ingestion pipeline

## Getting started

Download this notebook and open it locally using a Jupyter
server. Alternatively you may use Watson Studio.  To make using Watson
Studio easier we have provided a zip archive file containing the files
needed to complete this case study in Watson Studio. See the [Getting
started with Watson Studio](m1-u5-5-watson-studio.rst) page.

**You will need the following files to complete this case study**

* [m1-u6-case-study.ipynb](m1-u6-case-study.ipynb)
* [m1-u6-case-study-solution.ipynb](./notebooks/m1-u6-case-study-solution.ipynb)
* [aavail-customers.db](./data/aavail-customers.db)
* [aavail-steams.csv](./data/aavail-streams.csv)

1. Fill in all of the places in this notebook marked with ***YOUR CODE HERE*** or ***YOUR ANSWER HERE***

2. When you have finished the case study there will be a short quiz

You may review the rest of this content as part of the notebook, but
once you are ready to get started be ensure that you are working with
a *live* version either as part of Watson Studio or locally.

## Data Sources

The data you will be sourcing from is contained in two sources.

1. A database ([SQLite](https://www.sqlite.org/index.html)) of `customer` data

2. A [CSV file](https://en.wikipedia.org/wiki/Comma-separated_values) of `stream` level data

   >You will create a simple data pipeline that
   (1) simplifies the data for future analysis

   (2) performs quality assurance checks.

The process of building *the data ingestion pipeline* entails
extracting data, transforming it, and loading it into an appropriate
data storage technology.  When constructing a pipeline it is important
to keep in mind that they generally process data in batches.  For
example, data may be compiled during the day and the batch could be
processed during the night.  The data pipeline may also be optimized
to execute as a streaming computation (i.e., every event is handled as
it occurs).

## PART 1: Gathering the data

The following is an [Entity Relationship Diagram
(ERD)](https://en.wikipedia.org/wiki/Entity%E2%80%93relationship_model)
that details the tables and contents of the database.

<img src="./images/aavail-schema.svg" alt="customer database schema" style="width: 600px;"/>

In [3]:
## all the imports you will need for this case study
import os
import pandas as pd
import numpy as np
import sqlite3

Much of the data exist in a database.  You can connect to is using the `sqlite3` Python package with the function shown below.  Note that is is good practice to wrap your connect functions in a [try-except statement](https://docs.python.org/3/tutorial/errors.html) to cleanly handle exceptions.

In [4]:
def connect_db(file_path):
    try:
        conn = sqlite3.connect(file_path)
        print("...successfully connected to db\n")
    except Error as e:
        print("...unsuccessful connection\n",e)
    
    return(conn)

In [5]:
## make the connection to the database
conn = connect_db('../data_asset/aavail-customers.db')

## print the table names
tables = [t[0] for t in conn.execute("SELECT name FROM sqlite_master WHERE type='table';")]
print(tables)

...successfully connected to db

['CUSTOMER', 'INVOICE', 'INVOICE_ITEM', 'COUNTRY']


### QUESTION 1:

**extract the relevant data from the DB**

Query the database and extract the following data into a [Pandas
DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).
 
* Customer ID (integer)
* Last name
* First name
* DOB
* City
* State
* Country (the name NOT the country_id)
* Gender

Remember that that SQL is case-insensitive, but it is traditional to
use ALL CAPS for SQL keywords. It is also a convention to end SQL
statements with a semi-colon.

#### Resources

* [W3 schools SQL tutorial](https://www.w3schools.com/sql)
* [W3 schools SQL joins](https://www.w3schools.com/sql/sql_join.asp)

In [33]:
# notice that there is no way to direcly see the results of your
# query. conn.execute returns a sqllite object. in order to view the
# results in python you should store the information of your query in
# the standard data formats you use in jupyter notebooks.

a = pd.DataFrame(conn.execute("SELECT * FROM CUSTOMER;"))

a.head()

cust_id = pd.DataFrame(
    conn.execute(
        """
        SELECT c.DOB, c.customer_id, c.last_name, 
               c.first_name, c.state, c.gender, COUNTRY.country_name
        FROM CUSTOMER as c
        JOIN COUNTRY ON c.country_id = COUNTRY.country_id;
        """
    )
)

cust_id.head()


,0,1,2,3,4,5,6
0,07/30/98,1,Todd,Kasen,South Carolina,m,united_states
1,04/12/89,2,Garza,Ensley,None,f,singapore
2,09/12/97,3,Carey,Lillian,Alabama,f,united_states
3,01/28/99,4,Christensen,Beau,New York,m,united_states
4,03/23/98,5,Gibson,Ernesto,None,m,singapore


In [107]:
cust_id.columns = ["dob", "cust_id", "last_name", "first_name", "state", "gender", "country_name"]

### QUESTION 2:

**Extract the relevant data from the CSV file**

For each ```customer_id``` determine if a customer has stopped their
subscription or not and save it in a dictionary or another data
container.

In [34]:
df_streams = pd.read_csv("../data_asset/aavail-streams.csv")
df_streams.head()

,customer_id,stream_id,date,invoice_item_id,subscription_stopped
0,1,1420.0,2018-10-21,2.0,0
1,1,1343.0,2018-10-23,2.0,0
2,1,1756.0,2018-11-05,2.0,0
3,1,1250.0,2018-11-06,2.0,0
4,1,1324.0,2018-11-12,2.0,0


In [44]:
## bool subscription stopped
bol = df_streams.subscription_stopped == 1

df_streams[bol].iloc[:,:-1]


,customer_id,stream_id,date,invoice_item_id
36,2,1726.0,2019-03-28,3.0
58,3,1453.0,2018-11-15,2.0
139,7,1469.0,2019-08-25,1.0
187,9,1309.0,2019-08-20,2.0
209,10,1272.0,2019-07-05,1.0
...,...,...,...,...
18399,975,1533.0,2019-03-28,1.0
18451,978,1356.0,2019-09-02,2.0
18488,980,1567.0,2019-01-16,2.0
18711,992,1613.0,2019-04-20,2.0


## PART 2: Checks for quality assurance

Sometimes it is known in advance which types of data integrity issues
to expect, but other times it is during the Exploratory Data Analysis
(EDA) process that these issues are identified.  After extracting data
it is important to include checks for quality assurance even on the
first pass through the AI workflow.  Here you will combine the data
into a single structure and provide a couple checks for quality
assurance.

### QUESTION 3: 

**Implement checks for quality assurance**

1. Remove any repeat customers based on ```customer_id```
2. Remove stream data that do not have an associated ```stream_id```
3. Check for missing values

In [105]:
# 1. Remove any repeat customers based on ```customer_id```
print(df_streams.shape)

pd.DataFrame.drop_duplicates(df_streams)

print(df_streams.shape)

# there are no general duplicates.
# if the task was to keep just unique customer_id

uniq = df_streams["customer_id"].unique()

# 2. Remove stream data that do not have an associated ```stream_id```

## check if there are null values
# df_streams[['stream_id']].isnull().values.any()

## get the null values
# df_streams[np.array(df_streams[['stream_id']].isnull())]

## remove the null values
bol = df_streams.subscription_stopped == 1
df_streams = df_streams[bol].iloc[:,:-1]
corrected = df_streams[np.array(df_streams[['stream_id']].notnull())]

# 3. Check for missing values
corrected.isnull().values.any()

## no more missing values



(18859, 5)
(18859, 5)


False

In [106]:
corrected

,customer_id,stream_id,date,invoice_item_id
36,2,1726.0,2019-03-28,3.0
58,3,1453.0,2018-11-15,2.0
139,7,1469.0,2019-08-25,1.0
187,9,1309.0,2019-08-20,2.0
209,10,1272.0,2019-07-05,1.0
...,...,...,...,...
18399,975,1533.0,2019-03-28,1.0
18451,978,1356.0,2019-09-02,2.0
18488,980,1567.0,2019-01-16,2.0
18711,992,1613.0,2019-04-20,2.0


### QUESTION 4: 

**combine the data into a single data structure**

For this example, the two most convenient structures for this task are
Pandas dataframes and NumPy arrays.  At a minimum ensure that your
structure accommodates the following.

1. A column for `customer_id`
2. A column for `country`
3. A column for ```age``` that is created from ```DOB```
4. A column ```customer_name``` that is created from ```first_name``` and ```last_name```
5. A column to indicate churn called ```is_subscriber```
7. A column that indicates ```subscriber_type``` that comes from ```invoice_item```
6. A column to indicate the total ```num_streams```

#### Resources

* [Python's datetime library](https://docs.python.org/3/library/datetime.html)
* [NumPy's datetime data type](https://docs.scipy.org/doc/numpy/reference/arrays.datetime.html)


In [119]:
from datetime import datetime

datetime_object = datetime.strptime(cust_id.dob[0], '%m/%d/%y')

datetime_now = datetime.today()

datetime_now.year - datetime_object.year


22

In [121]:
def age(x):
    datetime_object = datetime.strptime(x, '%m/%d/%y')

    datetime_now = datetime.today()

    return datetime_now.year - datetime_object.year
    

In [130]:
# map function
print(cust_id['dob'].apply (lambda row: age(row)))

cust_id['age'] = cust_id['dob'].apply (lambda row: age(row))

cust_id.head()


0       22
1       31
2       23
3       21
4       22
        ..
1002    39
1003    20
1004    38
1005    22
1006    50
Name: dob, Length: 1007, dtype: int64


,dob,cust_id,last_name,first_name,state,gender,country_name,age
0,07/30/98,1,Todd,Kasen,South Carolina,m,united_states,22
1,04/12/89,2,Garza,Ensley,None,f,singapore,31
2,09/12/97,3,Carey,Lillian,Alabama,f,united_states,23
3,01/28/99,4,Christensen,Beau,New York,m,united_states,21
4,03/23/98,5,Gibson,Ernesto,None,m,singapore,22


In [138]:
cust_id['name'] = cust_id.first_name + " " + cust_id.last_name

cust_id.head()

,dob,cust_id,last_name,first_name,state,gender,country_name,age,name
0,07/30/98,1,Todd,Kasen,South Carolina,m,united_states,22,Kasen Todd
1,04/12/89,2,Garza,Ensley,None,f,singapore,31,Ensley Garza
2,09/12/97,3,Carey,Lillian,Alabama,f,united_states,23,Lillian Carey
3,01/28/99,4,Christensen,Beau,New York,m,united_states,21,Beau Christensen
4,03/23/98,5,Gibson,Ernesto,None,m,singapore,22,Ernesto Gibson


In [136]:
## YOUR CODE HERE
print(cust_id.columns)
print(corrected.columns)





Index(['dob', 'cust_id', 'last_name', 'first_name', 'state', 'gender',
       'country_name', 'age', 'name'],
      dtype='object')
Index(['customer_id', 'stream_id', 'date', 'invoice_item_id'], dtype='object')


## PART 3: Automating the process

To ensure that you code can be used to automate this process.  First
you will save you dataframe or numpy array as a CSV file.

### QUESTION 5:

**Take the initial steps towards automation**

1. Save your cleaned, combined data as a CSV file.

2. From the code above create a function or class that performs all of the steps given a database file and a streams CSV file.

3. Run the function in batches and write a check to ensure you got the same result that you did in the code above.

There will be some logic involved to ensure that you do not write the same data twice to the target CSV file.

Shown below is some code that will split your streams file into two batches. 

In [19]:
body = client_bfb337e9d3b24554ad9a816fe610a744.get_object(Bucket='aienterpriseworkflowmodule1casest-donotdelete-pr-qwt6qhnjsctfni',Key='aavail-streams.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

## code to split the streams csv into batches

df_all = pd.read_csv(body)
half = int(round(df_all.shape[0] * 0.5))
df_part1 = df_all[:half]
df_part2 = df_all[half:]
df_part1.to_csv = (os.path.join(".","aavail-streams-1.csv"))
df_part2.to_csv = (os.path.join(".","aavail-streams-2.csv"))

You will need to save your function as a file.  The following cell
demonstrates how to do this from within a notebook.

In [20]:
%%writefile aavail-data-ingestor.py
#!/usr/bin/env python

import os
import pandas as pd
import numpy as np
import sqlite3

data_dir = os.path.join(".","data")

pass

Writing aavail-data-ingestor.py


You will also need to be able to pass the file names to your function
without hardcoding them into the script itself.  This is an important
step towards automation.  Here are the two libraries commonly used to
accomplish this in Python.

* [getopt](https://docs.python.org/3/library/getopt.html)
* [argparse](https://docs.python.org/3/library/argparse.html)

You may run the script you just created from the commandline directly or from within this notebook using:

In [ ]:
!python aavail-data-ingestor.py aavail-customers.db aavail-streams-1.csv

Run the script once for each batch that you created and then load both
the original and batch versions back into the notebook to check that
they are the same.

In [13]:
## YOUR CODE HERE




### QUESTION 6:

**How can you improve the process?**

In paragraph form or using bullets write down some of the ways that you could improve this pipeline.

- stream the data. don't work in batches

- add some unit tests for checking that the new data are conform

- add some logging for dealing with the return cases.



In [148]:
print("\nCleaning Summary\n{}".format("-"*35))
romeo = pd.DataFrame({'a': [1,1,3], 'b' : [2,2,3]})

print(romeo)
print(romeo.duplicated())


Cleaning Summary
-----------------------------------
   a  b
0  1  2
1  1  2
2  3  3
0    False
1     True
2    False
dtype: bool
